# Statistical Analysis: NHST

## Data loading & pre-processing

In [2]:
import pandas as pd
import re

import analysis_utils as utils

Load data from all `../res` sub-directories (treatments):

In [3]:
summary_dfs = {}    # dict to store statistical summary data (<session_name>.json)
all_data_dfs = {}   # dict to store all individual measurements (all_data_<session_name>.json)

experiment_data_dir = '../res' # directory generated by "eval.py", contains the .json files with the experiment data

iteration_structure = True # flag to set the directory structure of the input data

# Load all the experiment data
summary_dfs, all_data_dfs = utils.load_experiment_data(experiment_data_dir, iteration_structure)

### Flatten GPU and VRAM columns 
These contains nested objects and values.
We are only interested in a subset of the contained data.

In [4]:
def flatten_gpu_vram_columns(df: pd.DataFrame) -> pd.DataFrame:
    # TODO: fix the key for extracting "vram_max_usage_mib" from "max_consumed_MiB" -> new format (check repo)
    
    # GPU columns
    df["gpu_util_mean"] = df["GPU"].apply(lambda x: x["utilization"]["avg"] if isinstance(x, dict) else None)
    df["gpu_util_max"]  = df["GPU"].apply(lambda x: x["utilization"]["max"] if isinstance(x, dict) else None)

    # VRAM columns
    df["vram_util_mean"]     = df["VRAM"].apply(lambda x: x["utilization"]["avg"] if isinstance(x, dict) else None)
    df["vram_util_max"]      = df["VRAM"].apply(lambda x: x["utilization"]["max"] if isinstance(x, dict) else None)
    df["vram_max_usage_mib"] = df["VRAM"].apply(lambda x: x["max_consumed_MiB"] if isinstance(x, dict) else None) # TODO: FIX KEY

    # Drop the redundant columns
    df = df.drop(columns=["GPU", "VRAM"])

    return df

### Convert data to long format

In [5]:
def convert_dict_dfs_to_long(dictionary_dataframes: dict[pd.DataFrame], metrics_of_interest: list[str]) -> pd.DataFrame:
    rows = [] # Temporary list to hold all constructed rows of data

    # Iterate through each treatment and corresponding data
    for treatment_name, df in dictionary_dataframes.items():

        # Extract the treatment factors (dataset is a blocked variable)
        model, quant, dataset = treatment_name.split("_")
        
        for i, row in df.iterrows():
            # Extract iteration from last dir in data_path (e.g. "01" from ".../01", etc.)
            match = re.search(r"/(\d{2})$", row["data_path"])
            iteration = int(match.group(1)) if match else i + 1 # TODO: should we include the fallback or let it fail?

            # Construct row dictionary
            row_data = {
                "treatment": treatment_name,
                "model": model, 
                "quantization": quant, 
                "dataset": dataset,
                "iteration": iteration
            }
        
            # Add relevant metrics
            for metric in metrics_of_interest:
                row_data[metric] = row[metric]

            rows.append(row_data)

    # Combine all the rows of data into a single long-format dataframe
    long_dataframe: pd.DataFrame = pd.DataFrame(rows)

    return long_dataframe

### Flatten columns and convert to long format to prepare for NHST analysis

In [ ]:
efficacy_metrics = [
    "accuracy", "recall", 
    "precision", "f1",
    "balanced_accuracy", 
    "specificity"
]

efficiency_metrics = [
    "time_to_analyze",
    "gpu_util_mean", "gpu_util_max",
    "vram_util_mean", "vram_util_max",
    "vram_max_usage_mib"
]

# Flatten and extract the relevant GPU and VRAM metrics into distinct columns (they are intially nested objects)
flattened_dfs = {key: flatten_gpu_vram_columns(df) for key, df in all_data_dfs.items()}

# Convert data to long format; group by research question
long_df_efficacy = convert_dict_dfs_to_long(flattened_dfs, efficacy_metrics)
long_df_efficiency = convert_dict_dfs_to_long(flattened_dfs, efficiency_metrics)

# Sort the dataframes using custom ordering schemes
# Note: we can apply custom schemes to all the sorting columns if needed
sorting_order = ["dataset", "iteration", "model", "quantization"]
quant_order = ["NONE", "AWQ", "GPTQ", "AQLM"]
#long_df_efficacy["quantization"] = pd.Categorical(long_df_efficacy["quantization"], categories=quant_order, ordered=True)
#long_df_efficacy = long_df_efficacy.sort_values(by=sorting_order).reset_index(drop=True)

print(long_df_efficacy.head())
#print(long_df_efficiency.head())


      treatment model quantization dataset  iteration  accuracy  recall  \
0  MIS_AWQ_BTHS   MIS          AWQ    BTHS          1  0.850000    0.10   
1  MIS_AWQ_BTHS   MIS          AWQ    BTHS          2  0.850000    0.10   
2  MIS_AWQ_BTHS   MIS          AWQ    BTHS          3  0.850000    0.10   
3  MIS_AWQ_BTHS   MIS          AWQ    BTHS          4  0.841667    0.05   
4  MIS_AWQ_BTHS   MIS          AWQ    BTHS          5  0.841667    0.05   

   precision        f1  balanced_accuracy  specificity  
0        1.0  0.181818           0.923729     0.847458  
1        1.0  0.181818           0.923729     0.847458  
2        1.0  0.181818           0.923729     0.847458  
3        1.0  0.095238           0.920168     0.840336  
4        1.0  0.095238           0.920168     0.840336  


---

## Testing ANOVA assumptions

#### Big questions:

- do we use repeated-measures (RM) ANOVA or RM MANOVA (multivariate ANOVA)?
  - this choice depends on how we want to treat the DVs (dependent variables);
  - i.e., do we group efficacy metrics since they are interconnected?
- alternatively, what about using the Friedman test?


Option:
- MANOVA for RQ1: the confusion matrix metrics are interconnected
- ANOVA for RQ2: these metrics are independent
  - we simplify to just: time, max vram usage

Or: 
- Repeated Measures ANOVA for RQ1
- ANOVA for RQ2: same simplified dependent variables

Or:
- Friedman test for RQ1
- Friedman test for RQ2 with simplified DVs


In [ ]:
import numpy as np
from scipy.stats import shapiro, levene

### Univariate ANOVA assumption checks:

Shapiro-Wilks test (normality assumption)

In [45]:
def check_anova_normality(df: pd.DataFrame, metrics: list[str], print_results = True) -> dict:
    results = {} # Shapiro test results
    alpha = 0.05 # Significance level

    # Check each metric individually
    for metric in metrics:
        results[metric] = []

        if print_results: print(f"\nChecking assumptions for {metric.upper()}:")

        # Group by treatment
        for treatment in df["treatment"].unique():
            subset = df[df["treatment"] == treatment][metric]
            
            # Shapiro-Wilk test
            w, p = shapiro(subset)
            results[metric].append({treatment: [{"W": w, "p_value": p}]})

            # Print the result
            if print_results: 
                print(f"  Treatment: {treatment}")
                print(f"  Shapiro-Wilk p = {p:.4f} --> {'Pass' if p > alpha else 'Fail'} (normality)")

    return results

Check normality for RQ1: efficacy metrics

In [ ]:
_ = check_anova_normality(long_df_efficacy, efficacy_metrics)


Checking assumptions for ACCURACY:
  Treatment: MIS_AWQ_BTHS
  Shapiro-Wilk p = 0.0000 --> Fail (normality)
  Treatment: MIS_AWQ_ENCO
  Shapiro-Wilk p = 0.0021 --> Fail (normality)
  Treatment: MIS_AWQ_SNAKE
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_BTHS
  Shapiro-Wilk p = 0.0000 --> Fail (normality)
  Treatment: MIS_NONE_ENCO
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_SNAKE
  Shapiro-Wilk p = 1.0000 --> Pass (normality)

Checking assumptions for RECALL:
  Treatment: MIS_AWQ_BTHS
  Shapiro-Wilk p = 0.0000 --> Fail (normality)
  Treatment: MIS_AWQ_ENCO
  Shapiro-Wilk p = 0.0002 --> Fail (normality)
  Treatment: MIS_AWQ_SNAKE
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_BTHS
  Shapiro-Wilk p = 0.0000 --> Fail (normality)
  Treatment: MIS_NONE_ENCO
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_SNAKE
  Shapiro-Wilk p = 1.0000 --> Pass (normality)

Checking assumptions for PRECISION:
  Treat

c:\Users\Erik\Documents\University\Year-3\BSc-Thesis\Repositories\Upstream REST-at\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Check normality for RQ2: efficiency metrics

In [48]:
simple_efficiency_metrics = [
    "time_to_analyze",
    "vram_max_usage_mib"
]

_ = check_anova_normality(long_df_efficiency, simple_efficiency_metrics)


Checking assumptions for TIME_TO_ANALYZE:
  Treatment: MIS_AWQ_BTHS
  Shapiro-Wilk p = 0.0164 --> Fail (normality)
  Treatment: MIS_AWQ_ENCO
  Shapiro-Wilk p = 0.8446 --> Pass (normality)
  Treatment: MIS_AWQ_SNAKE
  Shapiro-Wilk p = 0.7515 --> Pass (normality)
  Treatment: MIS_NONE_BTHS
  Shapiro-Wilk p = 0.0461 --> Fail (normality)
  Treatment: MIS_NONE_ENCO
  Shapiro-Wilk p = 0.5152 --> Pass (normality)
  Treatment: MIS_NONE_SNAKE
  Shapiro-Wilk p = 0.0689 --> Pass (normality)

Checking assumptions for VRAM_MAX_USAGE_MIB:
  Treatment: MIS_AWQ_BTHS
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_AWQ_ENCO
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_AWQ_SNAKE
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_BTHS
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_ENCO
  Shapiro-Wilk p = 1.0000 --> Pass (normality)
  Treatment: MIS_NONE_SNAKE
  Shapiro-Wilk p = 1.0000 --> Pass (normality)


c:\Users\Erik\Documents\University\Year-3\BSc-Thesis\Repositories\Upstream REST-at\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
